In [ ]:
!pip install -U transformers torch pydantic==1.10.13

Define Core Functionality and Simplified Agents

In [ ]:
import json
import os
import time
import uuid
from datetime import datetime, timedelta
from typing import Any, Callable, Dict, List, Optional
from pydantic import BaseModel
from transformers import pipeline

def patient_id_uu():
    return str(uuid.uuid4().hex)

class Agent:
    def __init__(self, agent_name: str, system_prompt: str, model_name: str):
        self.agent_name = agent_name
        self.system_prompt = system_prompt
        self.model = pipeline("text2text-generation", model=model_name)

    def run(self, task: str):
        prompt = f"{self.system_prompt}\n\nPatient Case: {task}\n\nResponse:"
        output = self.model(prompt, max_length=512, num_return_sequences=1, pad_token_id=self.model.tokenizer.pad_token_id)
        return output[0]['generated_text']

# Initialize agents with a smaller open-source LLM
model_name = "google/flan-t5-base"
chief_medical_officer = Agent(
    agent_name="Chief Medical Officer",
    system_prompt="""
    You are the Chief Medical Officer coordinating a team of medical specialists for viral disease diagnosis.
    Your responsibilities include:
    - Gathering initial patient symptoms and medical history
    - Coordinating with specialists to form differential diagnoses
    - Synthesizing different specialist opinions into a cohesive diagnosis
    - Ensuring all relevant symptoms and test results are considered
    - Making final diagnostic recommendations
    - Suggesting treatment plans based on team input
    - Identifying when additional specialists need to be consulted
    - For each diferrential diagnosis provide minimum lab ranges to meet that diagnosis or be indicative of that diagnosis minimum and maximum

    Format all responses with clear sections for:
    - Initial Assessment (include preliminary ICD-10 codes for symptoms)
    - Differential Diagnoses (with corresponding ICD-10 codes)
    - Specialist Consultations Needed
    - Recommended Next Steps

    """,
    model_name=model_name,
)
internist = Agent(
    agent_name="Internist",
    system_prompt="""
    You are an Internal Medicine specialist responsible for comprehensive evaluation.

    For each case, provide:

    Clinical Assessment:
    - System-by-system review
    - Vital signs analysis
    - Comorbidity evaluation

    Medical Coding:
    - ICD-10 codes for:
        * Primary conditions
        * Secondary diagnoses
        * Complications
        * Chronic conditions
        * Signs and symptoms
    - Include hierarchical condition category (HCC) codes where applicable

    Document supporting evidence for each code selected.
    """,
    model_name=model_name,
)
medical_coder = Agent(
    agent_name="Medical Coder",
    system_prompt="""
    You are a highly experienced and certified medical coder with extensive knowledge of ICD-10 coding guidelines, clinical documentation standards, and compliance regulations. Your responsibility is to ensure precise, compliant, and well-documented coding for all clinical cases.

    ### Primary Responsibilities:
    1. **Review Clinical Documentation**: Analyze all available clinical records, including specialist inputs, physician notes, lab results, imaging reports, and discharge summaries.
    2. **Assign Accurate ICD-10 Codes**: Identify and assign appropriate codes for primary diagnoses, secondary conditions, symptoms, and complications.
    3. **Ensure Coding Compliance**: Follow the latest ICD-10-CM/PCS coding guidelines, payer-specific requirements, and organizational policies.
    4. **Document Code Justification**: Provide clear, evidence-based rationale for each assigned code.

    ### Detailed Coding Process:
    - **Review Specialist Inputs**: Examine all relevant documentation to capture the full scope of the patient's condition and care provided.
    - **Identify Diagnoses**: Determine the primary and secondary diagnoses, as well as any symptoms or complications, based on the documentation.
    - **Assign ICD-10 Codes**: Select the most accurate and specific ICD-10 codes for each identified diagnosis or condition.
    - **Document Supporting Evidence**: Record the documentation source (e.g., lab report, imaging, or physician note) for each code to justify its assignment.
    - **Address Queries**: Note and flag any inconsistencies, missing information, or areas requiring clarification from providers.

    ### Output Requirements:
    Your response must be clear, structured, and compliant with professional standards. Use the following format:

    1. **Primary Diagnosis Codes**:
        - **ICD-10 Code**: [e.g., E11.9]
        - **Description**: [e.g., Type 2 diabetes mellitus without complications]
        - **Supporting Documentation**: [e.g., Physician's note dated MM/DD/YYYY]

    2. **Secondary Diagnosis Codes**:
        - **ICD-10 Code**: [Code]
        - **Description**: [Description]
        - **Order of Clinical Significance**: [Rank or priority]

    3. **Symptom Codes**:
        - **ICD-10 Code**: [Code]
        - **Description**: [Description]

    4. **Complication Codes**:
        - **ICD-10 Code**: [Code]
        - **Description**: [Description]
        - **Relevant Documentation**: [Source of information]

    5. **Coding Notes**:
        - Observations, clarifications, or any potential issues requiring provider input.

    ### Additional Guidelines:
    - Always prioritize specificity and compliance when assigning codes.
    - For ambiguous cases, provide a brief note with reasoning and flag for clarification.
    - Ensure the output format is clean, consistent, and ready for professional use.
    """,

    model_name=model_name,
)
synthesizer = Agent(
    agent_name="Diagnostic Synthesizer",
    system_prompt="""
    You are responsible for creating the final diagnostic and coding assessment.

    Synthesis Requirements:
    1. Integrate all specialist findings
    2. Reconcile any conflicting diagnoses
    3. Verify coding accuracy and completeness

    Final Report Sections:
    1. Clinical Summary
        - Primary diagnosis with ICD-10
        - Secondary diagnoses with ICD-10
        - Supporting evidence
    2. Coding Summary
        - Complete code list with descriptions
        - Code hierarchy and relationships
        - Supporting documentation
    3. Recommendations
        - Additional testing needed
        - Follow-up care
        - Documentation improvements needed

    Include confidence levels and evidence quality for all diagnoses and codes.
    """,
    model_name=model_name,
)
summarizer_agent = Agent(
    agent_name="Condensed Summarization Agent",
    system_prompt="""
    You are an expert in creating concise and actionable summaries from tweets, short texts, and small reports. Your task is to distill key information into a compact and digestible format while maintaining clarity and context.

    ### Summarization Goals:
    1. Identify the most critical message or insight from the input text.
    2. Present the summary in a clear, concise format suitable for quick reading.
    3. Retain important context and actionable elements while omitting unnecessary details.

    ### Output Structure:
    #### 1. Key Insight:
    - **Main Point**: Summarize the core message in one to two sentences.
    - **Relevant Context**: Include key supporting details (if applicable).

    #### 2. Actionable Takeaways (if needed):
    - Highlight any recommended actions, important next steps, or notable implications.

    ### Guidelines for Summarization:
    - **Brevity**: Summaries should not exceed 280 characters unless absolutely necessary.
    - **Clarity**: Avoid ambiguity or technical jargon; focus on accessibility.
    - **Relevance**: Include only the most impactful information while excluding redundant or minor details.
    - **Tone**: Match the tone of the original content (e.g., professional, casual, or informative).

    ### Example Workflow:
    1. Analyze the input for the primary message or intent.
    2. Condense the content into a clear, actionable summary.
    3. Format the output to ensure readability and coherence.

    ### Output Style:
    - Clear, concise, and easy to understand.
    - Suitable for social media or quick report overviews.
    """,

    model_name=model_name,
)
treatment_agent = Agent(
    agent_name="Treatment-Agent",
    system_prompt="""
    You are a specialist in treatment options, responsible for recommending the most effective and cost-efficient treatments for patients, considering both traditional and modern medicine approaches.

    Primary Responsibilities:
    1. Provide treatment recommendations for various diagnoses
    2. Offer multiple treatment methods, including traditional and modern medicine approaches
    3. Rank treatment options based on effectiveness and estimated cost
    4. Consider patient-specific factors, such as age, health status, and allergies
    5. Provide detailed treatment plans, including dosages, frequencies, and duration

    For each case, provide:

    Treatment Recommendations:
    - Multiple treatment options, including traditional and modern medicine approaches
    - Ranking of treatment options based on effectiveness and estimated cost
    - Consideration of patient-specific factors, such as age, health status, and allergies

    Treatment Details:
    - Detailed treatment plans, including dosages, frequencies, and duration
    - Information on potential side effects and interactions
    - Monitoring and follow-up requirements

    Cost Analysis:
    - Estimated cost of each treatment option
    - Breakdown of costs, including medication, hospitalization, and other expenses

    Patient Education:
    - Clear explanations of treatment options and their benefits
    - Instructions for self-care and lifestyle modifications
    - Addressing patient concerns and questions

    Output Format:
    1. Treatment Options
        - Ranked list of treatment options with effectiveness and cost analysis
        - Detailed treatment plans
    2. Patient Education
        - Clear explanations of treatment options and their benefits
        - Instructions for self-care and lifestyle modifications
    3. Cost Analysis
        - Estimated cost of each treatment option
        - Breakdown of costs
    4. Monitoring and Follow-up
        - Requirements for monitoring and follow-up care
        - Scheduling and frequency of follow-up appointments

    Always specify:
    - Evidence-based information to support treatment recommendations
    - Consideration of patient preferences and values
    - Alternative treatment options for patients with specific needs or restrictions
    """,
    model_name=model_name,
)

agents = [medical_coder, synthesizer, treatment_agent]

Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu


In [ ]:
class MCSAgentOutputs(BaseModel):
    agent_id: Optional[str] = str(uuid.uuid4().hex)
    agent_name: Optional[str] = None
    agent_output: Optional[str] = None
    timestamp: Optional[str] = time.strftime("%Y-%m-%d %H:%M:%S")

class MCSOutput(BaseModel):
    run_id: Optional[str] = str(uuid.uuid4().hex)
    patient_id: Optional[str]
    agent_outputs: Optional[List[MCSAgentOutputs]] = None
    summary: Optional[str]
    timestamp: Optional[str] = time.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
class MedicalCoder:
    """
    Class to represent a medical coding diagnosis using open-source LLM.
    """
    def __init__(
        self,
        name: str = "Medical-coding-diagnosis",
        description: str = "Comprehensive medical diagnosis and coding system",
        agents: list = agents,
        patient_id: str = "001",
        max_loops: int = 1,
        output_folder_path: str = "reports",
        patient_documentation: str = None,
        agent_outputs: list = any,
        user_name: str = "User",
        summarization: bool = False,
        *args,
        **kwargs,
    ):
        self.name = name
        self.description = description
        self.agents = agents
        self.patient_id = patient_id
        self.max_loops = max_loops
        self.output_folder_path = output_folder_path
        self.patient_documentation = patient_documentation
        self.agent_outputs = []
        self.user_name = user_name
        self.summarization = summarization
        self.patient_id = patient_id_uu()
        self.output_file_path = (
            f"medical_diagnosis_report_{self.patient_id}.md",
        )
        self.output_schema = MCSOutput(
            patient_id=self.patient_id, agent_outputs=[], summary=""
        )

    def run(self, task: str = None, img: str = None, *args, **kwargs):
        """Runs the medical coding and diagnosis system."""
        print("Running the medical coding and diagnosis system.")
        try:
            case_info = f"Patient Information: {self.patient_id} \n Timestamp: {datetime.now()} \n Patient Documentation: {self.patient_documentation} \n Task: {task} "

            medical_coder_output = medical_coder.run(case_info)
            self.output_schema.agent_outputs.append(
                MCSAgentOutputs(
                    agent_name=medical_coder.agent_name,
                    agent_output=medical_coder_output,
                )
            )

            synthesizer_input = f"From {medical_coder.agent_name}: {medical_coder_output}"
            synthesizer_output = synthesizer.run(synthesizer_input)
            self.output_schema.agent_outputs.append(
                MCSAgentOutputs(
                    agent_name=synthesizer.agent_name,
                    agent_output=synthesizer_output,
                )
            )

            treatment_agent_input = f"From {synthesizer.agent_name}: {synthesizer_output}"
            treatment_agent_output = treatment_agent.run(treatment_agent_input)
            self.output_schema.agent_outputs.append(
                MCSAgentOutputs(
                    agent_name=treatment_agent.agent_name,
                    agent_output=treatment_agent_output,
                )
            )

            if self.summarization is True:
                output = summarizer_agent.run(treatment_agent_output)
                self.output_schema.summary = output

            return self.output_schema.json(indent=4)

        except Exception as e:
            print(f"An error occurred during the diagnosis process: {e}")

    def to_dict(self) -> Dict[str, Any]:
        """Converts the object to a dictionary."""
        return {k: str(v) for k, v in self.__dict__.items()}

    def save_patient_data(self, patient_id: str, case_data: str):
        """Saves patient data to a local JSON file."""
        try:
            data = {
                "patient_id": patient_id,
                "case_data": case_data,
                "timestamp": datetime.now().isoformat(),
            }
            with open(f"{patient_id}_report.json", "w") as file:
                json.dump(data, file, indent=4)
            print(f"Patient data saved for ID: {patient_id}")
        except Exception as e:
            print(f"Error saving patient data: {e}")

In [ ]:
# Example Usage
if __name__ == "__main__":
    patient_documentation = "Patient presents with ADHD, has been taking atomoxetine 18mg two times per day, still not able to improve the lifestyle and get a job"
    diagnosis = MedicalCoder(patient_documentation=patient_documentation, summarization=True)
    output = diagnosis.run(task="Diagnose the patient based on the provided symptoms.")
    print("\nDiagnosis Report:")
    print(output)

Token indices sequence length is longer than the specified maximum sequence length for this model (787 > 512). Running this sequence through the model will result in indexing errors


Running the medical coding and diagnosis system.


Token indices sequence length is longer than the specified maximum sequence length for this model (882 > 512). Running this sequence through the model will result in indexing errors



Diagnosis Report:
{
    "run_id": "97b4676f83a24bc483ecc5b40da5c42b",
    "patient_id": "e32f80b3a93140b8b00441ad3af93467",
    "agent_outputs": [
        {
            "agent_id": "402098c2aa11447b89cab1c0d9bdbe6a",
            "agent_name": "Medical Coder",
            "agent_output": "1. **Assign ICD-10 Codes**: Identify and assign appropriate codes for each identified diagnosis or condition. 2. Assign ICD-10 Codes**: Identify and assign appropriate codes for each identified diagnosis or condition. 3. Assign ICD-10 Codes**: Identify and assign appropriate codes for each identified diagnosis or condition. 4. Assign ICD-10 Codes**: Identify and assign appropriate codes for each identified diagnosis or condition. 5. Assign ICD-10 Codes**: Identify and assign appropriate codes for each identified diagnosis or condition. 6. Assign ICD-10 Codes**: Identify and assign appropriate codes for each identified diagnosis or condition. 7. Assign ICD-10 Codes**: Identify and assign appropriate co